Importación de modulos necesarios:

In [1]:
from datetime import datetime, date
from pathlib import Path
import pandas as pd

Declaración de paths de los archivos y Dataframes

In [2]:
BASE_DIR = Path('').parent
UCine_path = Path('GHUCine_select.csv')
UBsAs_path = Path('GHUBuenosAires_select.csv')

cine_df = pd.read_csv(BASE_DIR/UCine_path)
bs_df = pd.read_csv(BASE_DIR/UBsAs_path)

cp_df = pd.read_csv(BASE_DIR/"assets/codigos_postales.csv")

# Elimina los registro duplicados y toma el primero como valido.
cp_df.drop_duplicates(subset=['localidad'], keep='first', inplace=True)

print(cp_df)

      codigo_postal                     localidad
0              1601            ISLA MARTIN GARCIA
1              1602                       FLORIDA
2              1603                VILLA MARTELLI
3              1605                    CARAPACHAY
4              1607                 VILLA ADELINA
...             ...                           ...
1515           9409             ISLA GRAN MALVINA
1516           9410                BAHIA LAPATAIA
1517           9411  BASE AEREA TENIENTE MATIENZO
1518           9420             ASERRADERO ARROYO
1519           9421               FRIGORIFICO CAP

[1442 rows x 2 columns]


Transformaciones de los string:

- university: str minúsculas, sin espacios extras, ni guiones
- career: str minúsculas, sin espacios extras, ni guiones
- first_name: str minúscula y sin espacios, ni guiones
- last_name: str minúscula y sin espacios, ni guiones
- location: str minúscula sin espacios extras, ni guiones


In [3]:
def transform(df,txt_name):

    # Lista de columnas a transformar a string
    list_str = ['university','career','first_name','last_name','location', 'email']

    # Transformacion de las listas a string
    for column in list_str:
        df[column] = df[column].astype('string').str.replace('-', ' ').str.lower()

    # Transformacion de gender
    df['gender'] = df['gender'].astype('string').str.lower().replace({'f':'female', 'm':'male'}).astype('category')

    # Transformacion de fechas.
    def date_format(fecha):
        format = '%y-%b-%d'
        fecha = datetime.strptime(fecha, format)
        return fecha.date()
    
    try:
        df['inscription_date'] = df['inscription_date'].map(lambda date: date_format(date))
        df['inscription_date'] = pd.to_datetime(df['inscription_date'], yearfirst=True)
        #Poner to_datetime
    except:
        df['inscription_date'] = pd.to_datetime(df['inscription_date'], dayfirst=True)
    

    #Birth_date: (para calcular las edades)
    #Criterio utilizado en fechas con formato corto:
    #si es <= 07 le ponemos 20 y sino 19.
    #En el caso con fechas que contengan yyyy solo se formatea a: %Y-%m-%d 
    
    # cambiar por un .apply()
    df['birth_date'] = df['birth_date'].map(lambda date:date if len(date) > 9 else '19' + date if int(date[0:2]) >= 7 else '20' + date)
    try:
        df['birth_date'] = pd.to_datetime(df['birth_date'], dayfirst=True)
    except:
        df['birth_date']
    
    # calculo de edad:
    df['age'] =  (df['inscription_date'] - df['birth_date']).dt.days # devuelve una serie con los dias en formato int64
    df['age'] = df['age']/365 # transformado en año diviendo por el año
    df['age'] = df['age'].map(lambda age:round(age)).astype(int) # convirtiendo a entero

    # Postal_code y location (Según corresponda)
    if df['location'].isnull().values.any():
        print('llenamos location')

    else:
        print('llenamos postal_code')

    df.to_csv(f'{txt_name}.txt', header=True, index=None, sep=' ', mode='w')

    return df

transform(bs_df,'GHUBuenosAires_dataset')

#cp_df.set_index('codigo_postal').to_dict()['localidad'][1603]


llenamos location


,university,career,inscription_date,first_name,last_name,birth_date,gender,age,postal_code,location,email
0,universidad de buenos aires,actuario,2020-09-24,<NA>,amy wallace,1921-03-12,female,100,8431,<NA>,brookejones@yahoo.com
1,universidad de buenos aires,licenciatura en gestión de agroalimentos,2020-09-27,<NA>,angela hudson,2005-02-21,female,16,3151,<NA>,cpatterson@yahoo.com
2,universidad de buenos aires,medicina,2020-10-18,<NA>,jessica sandoval,1945-09-12,female,75,6601,<NA>,melinda04@hotmail.com
3,universidad de buenos aires,licenciatura en ciencias antropológicas,2020-09-20,<NA>,david cooper,1929-08-01,male,91,6105,<NA>,lukegutierrez@yahoo.com
4,universidad de buenos aires,licenciatura en kinesiología y fisiatría,2020-12-12,<NA>,benjamin nicholson,1968-06-05,male,53,5345,<NA>,bgonzalez@hotmail.com
...,...,...,...,...,...,...,...,...,...,...,...
1839,universidad de buenos aires,arquitectura,2020-12-05,<NA>,lisa brown,1915-10-07,female,105,8183,<NA>,fnorman@hotmail.com
1840,universidad de buenos aires,medicina,2020-10-17,<NA>,steven lewis,1955-01-04,male,66,4413,<NA>,maria76@hotmail.com
1841,universidad de buenos aires,ingeniería mecánica,2020-10-18,<NA>,donna bailey,1935-02-26,female,86,6600,<NA>,gilbertmichele@yahoo.com
1842,universidad de buenos aires,ingeniería mecánica,2020-12-11,<NA>,victoria valdez,1965-07-03,female,55,3441,<NA>,robert61@hotmail.com


gender: str choice(male, female)